In [9]:
import sagemaker
import boto3 
from sagemaker.image_uris import retrieve
from sagemaker.pytorch import PyTorch, PyTorchModel
import os 
from PIL import Image
from torchvision import transforms
import json

In [10]:
sm_boto3 = boto3.client("sagemaker")

sess = sagemaker.Session()

region = sess.boto_session.region_name

In [11]:
# train_data = {'train': 's3://pitahaya/Augmented Dataset/'}
# model_output = 's3://pitahaya/output/'
# model_artifact='s3://pitahaya/output/pytorch-training-2024-11-19-05-56-45-204/output/model.tar.gz'

train_data = os.getenv("TRAINING_PATH")
model_output = os.getenv("OUTPUT_PATH")
model_artifact = os.getenv("ARTIFACT")
role_arn = os.getenv("ROLE_ARN")

In [12]:
train_data, model_artifact, model_output, role_arn

('s3://pitahaya/Augmented Dataset/',
 's3://pitahaya/output/pytorch-training-2024-11-19-05-56-45-204/output/model.tar.gz',
 's3://pitahaya/output/',
 'arn:aws:iam::985570999006:role/service-role/AmazonSageMaker-ExecutionRole-20240718T155914')

In [13]:

if not train_data and model_output and model_artifact and role_arn:
    
    raise ValueError("Las variables de entorno de SAGEMAKER no estan configuradas.")

else: print("rol cargado exitosamente")

rol cargado exitosamente


In [14]:
hyperparameters={
        'epochs': 10,
        'batch_size': 32,
        'learning_rate': 0.001
    }

In [15]:
estimator = PyTorch(
    entry_point='train.py',  
    role=role_arn,
    instance_count=1,  
    instance_type='ml.m5.large', 
    framework_version='1.9.0', 
    py_version='py38',  
    hyperparameters=hyperparameters,
    # model_data=model_artifact,
    output_path=model_output,  
    sagemaker_session=sess
)

In [ ]:

# estimator.fit(train_data)
estimator.fit(train_data)


[04/10/25 10:59:01] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=298531;file:///home/dev/Documents/Github/pitahaya/.venv/lib/python3.13/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=904184;file:///home/dev/Documents/Github/pitahaya/.venv/lib/python3.13/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     image_uri is not presented, retrieving image_uri based on            ]8;id=954168;file:///home/dev/Documents/Github/pitahaya/.venv/lib/python3.13/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=832036;file:///home/dev/Documents/Github/pitahaya/.venv/lib/python3.13/site-packages/sagemaker/image_uris.py#681\681]8;;\
                             instance_type, framework etc.                                                         

                    INFO     image_uri is not presented, retrieving image_uri based on            ]8;id=266295;file:///home/dev/Documents/Github/pitahaya/.venv/lib/python3.13/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=789383;file:///home/dev/Documents/Github/pitahaya/.venv/lib/python3.13/site-packages/sagemaker/image_uris.py#681\681]8;;\
                             instance_type, framework etc.                                                         

                    INFO     Creating training-job with name:                                       ]8;id=436038;file:///home/dev/Documents/Github/pitahaya/.venv/lib/python3.13/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=673628;file:///home/dev/Documents/Github/pitahaya/.venv/lib/python3.13/site-packages/sagemaker/session.py#1042\1042]8;;\
                             pytorch-training-2025-04-10-15-59-01-231                                              

2025-04-10 15:59:04 Starting - Starting the training job...
2025-04-10 15:59:18 Starting - Preparing the instances for training...

In [7]:
model = PyTorchModel(
    model_data=model_artifact,
    role=role_arn,
    framework_version='1.13',
    py_version='py39',  # Cambia esto si usaste otra versión
)

In [8]:
# Desplegar el modelo
predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.m4.xlarge',  # Tipo de instancia para inferencias
)


--------!

In [9]:
endpoint_name = "pytorch-inference-2024-11-19-15-07-51-686"  # Reemplaza con el nombre de tu endpoint

# Crear cliente de SageMaker Runtime
sagemaker_runtime = boto3.client('sagemaker-runtime')

In [12]:
image_path = 'Augmented_Dataset/normal/Fresh_Dragon_Augmented_Data0003.jpg'
image = Image.open(image_path)
image = image.resize((224, 224))  


In [14]:
transform = transforms.Compose([
    transforms.ToTensor(),  # Convertir a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalización estándar
])


In [15]:
input_tensor = transform(image).unsqueeze(0)  # Añadir dimensión de batch
# Convertir tensor a JSON para enviarlo al endpoint
payload = json.dumps(input_tensor.numpy().tolist())

In [16]:
# Hacer la predicción llamando al endpoint
response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",  # Tipo de contenido del payload
    Body=payload
)

KeyboardInterrupt: 

In [ ]:
result = json.loads(response['Body'].read().decode())
print(f"Resultado de la predicción: {result}")

In [17]:
predictor.delete_endpoint()